# Imports

In [32]:
# General
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Hyperparameter tuning
import optuna

# Models
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier


# Data processing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Metrics
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

# Data preparation

In [38]:
# Data input
data = pd.read_csv('../data/train.csv')
data_to_predict = pd.read_csv('../data/test.csv')
data_ccrisk = pd.read_csv('../data/credit_risk_dataset.csv')
data_ccrisk.dropna(inplace=True)
data_ccrisk.drop_duplicates(inplace=True)

X_old = data.drop(['loan_status'], axis=1)
y_old = data['loan_status']

# Merge the dataframes
data_no_id = data.drop(['id'], axis=1)
merged_data = pd.concat([data_no_id, data_ccrisk], ignore_index=True)

X = merged_data.drop(['loan_status'], axis=1)
y = merged_data['loan_status']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, shuffle=True, test_size=0.2, random_state=2024)

In [39]:
# Data processing

categorical_columns = X.select_dtypes(include=['object']).columns

categorical_ordinal = ['loan_grade']
categorical_onehot = categorical_columns.drop(categorical_ordinal)

numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns

# Preprocessor
preprocessor = ColumnTransformer(
	transformers=[
		('ordinal', OrdinalEncoder(), categorical_ordinal),
		('onehot', OneHotEncoder(), categorical_onehot),
		('scaler', StandardScaler(), numerical_columns)
	])

preprocessor.fit(X_train)

X_train_prep = preprocessor.transform(X_train)
X_test_prep = preprocessor.transform(X_test)

# Models

## XGBoost

In [6]:
def xgb_objective(trial):
    # Hyperparameter search space
    param = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 0.5),
        'lambda': trial.suggest_float('lambda', 1e-8, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 10.0, log=True),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 3.0),
        'eval_metric': 'auc',  # AUC is the metric
        'objective': 'binary:logistic'  # Use binary logistic, probability outputs
    }

    # Initialize the model
    model = xgb.XGBClassifier(**param)

    # Fit the model
    model.fit(X_train_prep, y_train, 
              eval_set=[(X_test_prep, y_test)],  
              verbose=False)

    # Predict proba for validation set
    y_pred_prob = model.predict_proba(X_test_prep)[:, 1]

    auc = roc_auc_score(y_test, y_pred_prob)
    
    return auc

# Study object to run the optimization. I want to maximize AUC
xgb_study = optuna.create_study(direction='maximize')
xgb_study.optimize(xgb_objective, n_trials=50)


print(f"Best trial: {xgb_study.best_trial.params}")

[I 2024-10-09 22:05:58,334] A new study created in memory with name: no-name-44073533-931b-4bf9-a0e4-6b809fdcc510
[I 2024-10-09 22:06:01,537] Trial 0 finished with value: 0.9525810347956751 and parameters: {'max_depth': 8, 'learning_rate': 0.23759751295096465, 'n_estimators': 104, 'subsample': 0.6861085235631791, 'colsample_bytree': 0.6075327980311174, 'gamma': 0.12451478473153665, 'lambda': 1.5462291489103863e-06, 'alpha': 0.0014761980421400247, 'scale_pos_weight': 2.9166529779731913}. Best is trial 0 with value: 0.9525810347956751.
[I 2024-10-09 22:06:12,725] Trial 1 finished with value: 0.9548030154749317 and parameters: {'max_depth': 10, 'learning_rate': 0.10771767717037606, 'n_estimators': 374, 'subsample': 0.7953274560696989, 'colsample_bytree': 0.8337629717071376, 'gamma': 0.2995102374207908, 'lambda': 1.2907977806555766, 'alpha': 1.3013028313152889e-06, 'scale_pos_weight': 2.323853613045628}. Best is trial 1 with value: 0.9548030154749317.
[I 2024-10-09 22:06:20,947] Trial 2 fi

Best trial: {'max_depth': 9, 'learning_rate': 0.04443861243064845, 'n_estimators': 295, 'subsample': 0.9129336231196766, 'colsample_bytree': 0.7817900881520917, 'gamma': 0.20733828465084803, 'lambda': 0.0007763279982043077, 'alpha': 4.17984861400515, 'scale_pos_weight': 2.6715664022423775}


In [7]:
xgb_best_params = xgb_study.best_trial.params
xgb_best_score = xgb_study.best_trial.value
xgb_best_score

0.959733166826617

## CatBoostClassifier

In [8]:
from catboost import CatBoostClassifier

In [9]:
def cat_objective(trial):
    # Define the hyperparameter search space
    param = {
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'iterations': trial.suggest_int('iterations', 200, 1000),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True),
        'border_count': trial.suggest_int('border_count', 32, 255),  # CatBoost feature binning
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_float('random_strength', 0.0, 1.0),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 3.0),
        'eval_metric': 'AUC',  # Using AUC as the evaluation metric
        'logging_level': 'Silent',  # Disable CatBoost output
        'task_type': 'CPU',  # Use CPU (set 'GPU' if you have a GPU available)
        'use_best_model': True
    }

    # Initialize the CatBoost model with the trial's parameters
    model = CatBoostClassifier(**param)

    # Fit the model
    model.fit(X_train_prep, y_train,
              eval_set=[(X_test_prep, y_test)],
              early_stopping_rounds=20,  # Early stopping to prevent overfitting
              verbose=False)

    # Get predictions and calculate AUC score
    y_pred_prob = model.predict_proba(X_test_prep)[:, 1]  # Probabilities for class 1
    auc = roc_auc_score(y_test, y_pred_prob)

    return auc

# Create a study object and start the optimization process
cat_study = optuna.create_study(direction='maximize')  # We want to maximize AUC
cat_study.optimize(cat_objective, n_trials=50)

# Output the best trial
print(f"Best trial: {cat_study.best_trial.params}")

[I 2024-10-09 22:11:51,100] A new study created in memory with name: no-name-e72c9759-8c17-475d-a66c-405ea194cf53
[I 2024-10-09 22:11:58,468] Trial 0 finished with value: 0.9535014045536119 and parameters: {'depth': 7, 'learning_rate': 0.14900669887451898, 'iterations': 972, 'l2_leaf_reg': 2.832927821560049e-06, 'border_count': 164, 'bagging_temperature': 0.5177074086756733, 'random_strength': 0.3755586295324942, 'scale_pos_weight': 1.0033482236176379}. Best is trial 0 with value: 0.9535014045536119.
[I 2024-10-09 22:12:01,653] Trial 1 finished with value: 0.9435656688784282 and parameters: {'depth': 9, 'learning_rate': 0.1442107216774723, 'iterations': 403, 'l2_leaf_reg': 1.9701502555594304e-05, 'border_count': 91, 'bagging_temperature': 0.14870006576346828, 'random_strength': 0.12286492442004282, 'scale_pos_weight': 2.431117089849309}. Best is trial 0 with value: 0.9535014045536119.
[I 2024-10-09 22:12:11,152] Trial 2 finished with value: 0.9434425794273231 and parameters: {'depth': 

Best trial: {'depth': 6, 'learning_rate': 0.1377846993488999, 'iterations': 335, 'l2_leaf_reg': 1.9947064092180422, 'border_count': 223, 'bagging_temperature': 0.43873437022890877, 'random_strength': 0.3977615491895288, 'scale_pos_weight': 2.4634007576197803}


In [10]:
cat_best_params = cat_study.best_trial.params
cat_best_score = cat_study.best_trial.value
cat_best_score

0.9595616312456452

## LightGBM

In [12]:
# Objective function for LightGBM
def lgb_objective(trial):
    param = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'min_child_weight': trial.suggest_float('min_child_weight', 1e-3, 10.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
    }

    lgb_model = lgb.LGBMClassifier(**param)
    
    # Train the model
    lgb_model.fit(X_train_prep, y_train,
                  eval_set=[(X_test_prep, y_test)],
                  eval_metric='auc'
                  )

    # Predict and evaluate AUC
    y_pred_prob = lgb_model.predict_proba(X_test_prep)[:, 1]
    auc = roc_auc_score(y_test, y_pred_prob)

    return auc

# Optimize the objective function
lgb_study = optuna.create_study(direction='maximize')
lgb_study.optimize(lgb_objective, n_trials=50)

# Best parameters and AUC
print("Best LGBM Params: ", lgb_study.best_trial.params)
print("Best AUC for LGBM: ", lgb_study.best_value)

[I 2024-10-09 22:47:59,952] A new study created in memory with name: no-name-e4df7c3a-673f-47c0-a4ee-3f61b705cfdd


[LightGBM] [Info] Number of positive: 12366, number of negative: 60614
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004010 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 954
[LightGBM] [Info] Number of data points in the train set: 72980, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.169444 -> initscore=-1.589575
[LightGBM] [Info] Start training from score -1.589575
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

[I 2024-10-09 22:48:06,809] Trial 0 finished with value: 0.9605070990867199 and parameters: {'learning_rate': 0.20286923205339508, 'num_leaves': 268, 'max_depth': 3, 'min_child_samples': 64, 'min_child_weight': 5.427852638749153, 'subsample': 0.7080406522259487, 'colsample_bytree': 0.6149863635429692, 'n_estimators': 882}. Best is trial 0 with value: 0.9605070990867199.


[LightGBM] [Info] Number of positive: 12366, number of negative: 60614
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009931 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 954
[LightGBM] [Info] Number of data points in the train set: 72980, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.169444 -> initscore=-1.589575
[LightGBM] [Info] Start training from score -1.589575
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2024-10-09 22:48:10,991] Trial 1 finished with value: 0.9548021724695881 and parameters: {'learning_rate': 0.2920295183776929, 'num_leaves': 56, 'max_depth': 15, 'min_child_samples': 30, 'min_child_weight': 1.7023084792952516, 'subsample': 0.9259109299444939, 'colsample_bytree': 0.5835249499904873, 'n_estimators': 406}. Best is trial 0 with value: 0.9605070990867199.


...

[LightGBM] [Info] Number of positive: 12366, number of negative: 60614
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011911 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 954
[LightGBM] [Info] Number of data points in the train set: 72980, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.169444 -> initscore=-1.589575
[LightGBM] [Info] Start training from score -1.589575
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

[I 2024-10-09 22:52:31,914] Trial 48 finished with value: 0.9624465235825518 and parameters: {'learning_rate': 0.05625929357298218, 'num_leaves': 287, 'max_depth': 8, 'min_child_samples': 36, 'min_child_weight': 6.1290178895068035, 'subsample': 0.8069805580833423, 'colsample_bytree': 0.6288177374771741, 'n_estimators': 706}. Best is trial 46 with value: 0.9627544019391427.


[LightGBM] [Info] Number of positive: 12366, number of negative: 60614
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008586 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 954
[LightGBM] [Info] Number of data points in the train set: 72980, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.169444 -> initscore=-1.589575
[LightGBM] [Info] Start training from score -1.589575
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

[I 2024-10-09 22:52:41,071] Trial 49 finished with value: 0.9612736790597132 and parameters: {'learning_rate': 0.018850003105030495, 'num_leaves': 243, 'max_depth': 9, 'min_child_samples': 80, 'min_child_weight': 7.2272787228664805, 'subsample': 0.8714441381979217, 'colsample_bytree': 0.6501289447353497, 'n_estimators': 648}. Best is trial 46 with value: 0.9627544019391427.


Best LGBM Params:  {'learning_rate': 0.056732075773989935, 'num_leaves': 200, 'max_depth': 8, 'min_child_samples': 81, 'min_child_weight': 4.238151660690304, 'subsample': 0.8031135623721948, 'colsample_bytree': 0.6213074649038759, 'n_estimators': 530}
Best AUC for LGBM:  0.9627544019391427


In [13]:
lgb_best_params = lgb_study.best_trial.params
lgb_best_score = lgb_study.best_value
lgb_best_score

0.9627544019391427

## ExtraTreesClassifier

In [41]:
from sklearn.ensemble import ExtraTreesClassifier

def extratrees_objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_float('max_features', 0.1, 1.0)
    }

    model = ExtraTreesClassifier(**param)
    score = cross_val_score(model, X_train_prep, y_train, cv=3, scoring='roc_auc').mean()
    return score

extratrees_study = optuna.create_study(direction='maximize')
extratrees_study.optimize(extratrees_objective, n_trials=50, timeout=600)
print('Best parameters for ExtraTrees:', extratrees_study.best_params)

[I 2024-10-10 13:31:47,671] A new study created in memory with name: no-name-c32ecdb4-4771-4434-bace-0f8488b6a578
[I 2024-10-10 13:32:18,194] Trial 0 finished with value: 0.8976591052029291 and parameters: {'n_estimators': 226, 'max_depth': 6, 'min_samples_split': 10, 'min_samples_leaf': 6, 'max_features': 0.4734387050851664}. Best is trial 0 with value: 0.8976591052029291.
[I 2024-10-10 13:33:08,979] Trial 1 finished with value: 0.924814880140564 and parameters: {'n_estimators': 260, 'max_depth': 18, 'min_samples_split': 2, 'min_samples_leaf': 8, 'max_features': 0.4062967570915128}. Best is trial 1 with value: 0.924814880140564.
[I 2024-10-10 13:33:54,873] Trial 2 finished with value: 0.9249496750803043 and parameters: {'n_estimators': 175, 'max_depth': 11, 'min_samples_split': 8, 'min_samples_leaf': 10, 'max_features': 0.8095108685097703}. Best is trial 2 with value: 0.9249496750803043.
[I 2024-10-10 13:34:24,255] Trial 3 finished with value: 0.9052801723230512 and parameters: {'n_es

Best parameters for ExtraTrees: {'n_estimators': 397, 'max_depth': 20, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 0.9049580703915163}


In [42]:
extratrees_best_params = extratrees_study.best_params
extratrees_best_score = extratrees_study.best_value
extratrees_best_score

0.9301930211288746

## LogisticRegression

In [52]:
from sklearn.linear_model import LogisticRegression

def logreg_objective(trial):
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2', 'elasticnet'])
    param = {
        'penalty': penalty,
        'C': trial.suggest_float('C', 1e-4, 1e2, log=True),
        'solver': trial.suggest_categorical('solver', ['liblinear', 'saga']),
        'l1_ratio': trial.suggest_float('l1_ratio', 0, 1) if penalty == 'elasticnet' else None,
    }
    
    if penalty == 'elasticnet' and param['solver'] == 'liblinear':
        param['solver'] = 'saga'

    model = LogisticRegression(**param, max_iter=1000)
    score = cross_val_score(model, X_train_prep, y_train, cv=3, scoring='roc_auc').mean()
    return score

logreg_study = optuna.create_study(direction='maximize')
logreg_study.optimize(logreg_objective, n_trials=50, timeout=600)
print('Best parameters for Logistic Regression:', logreg_study.best_params)

[I 2024-10-10 13:55:31,215] A new study created in memory with name: no-name-07076243-9304-4a9b-9399-74d8874e57a9
[I 2024-10-10 13:56:18,437] Trial 0 finished with value: 0.8806748215145322 and parameters: {'penalty': 'elasticnet', 'C': 0.01390002803654353, 'solver': 'liblinear', 'l1_ratio': 0.21785270388856437}. Best is trial 0 with value: 0.8806748215145322.
[I 2024-10-10 13:56:55,498] Trial 1 finished with value: 0.8813745300886522 and parameters: {'penalty': 'elasticnet', 'C': 35.81210793007274, 'solver': 'saga', 'l1_ratio': 0.1563063849049523}. Best is trial 1 with value: 0.8813745300886522.
[I 2024-10-10 13:57:30,392] Trial 2 finished with value: 0.8813765597024469 and parameters: {'penalty': 'l1', 'C': 5.840322698850495, 'solver': 'saga'}. Best is trial 2 with value: 0.8813765597024469.
c:\Users\Denis\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  wa

Best parameters for Logistic Regression: {'penalty': 'l1', 'C': 0.07290758788748607, 'solver': 'saga'}


In [53]:
logreg_best_params = logreg_study.best_params
logreg_best_score = logreg_study.best_value
logreg_best_score

0.881427941668749

## HistGradientBoostingClassifier

In [34]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

def histgb_objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'max_iter': trial.suggest_int('max_iter', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 10, 50),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 10, 50),
        'l2_regularization': trial.suggest_float('l2_regularization', 1e-5, 1.0, log=True)
    }

    model = HistGradientBoostingClassifier(**param)
    score = cross_val_score(model, X_train_prep, y_train, cv=3, scoring='roc_auc').mean()
    return score

histgb_study = optuna.create_study(direction='maximize')
histgb_study.optimize(histgb_objective, n_trials=50, timeout=600)
print('Best parameters for HistGradientBoosting:', histgb_study.best_params)


[I 2024-10-10 13:18:49,634] A new study created in memory with name: no-name-f590faca-58f6-491e-a758-160cd3cd9448
[I 2024-10-10 13:19:10,465] Trial 0 finished with value: 0.9442874793983109 and parameters: {'learning_rate': 0.01770656600858437, 'max_iter': 341, 'max_depth': 20, 'min_samples_leaf': 35, 'max_leaf_nodes': 17, 'l2_regularization': 1.728789982584076e-05}. Best is trial 0 with value: 0.9442874793983109.
[I 2024-10-10 13:19:18,401] Trial 1 finished with value: 0.9403395113266696 and parameters: {'learning_rate': 0.04339927037290552, 'max_iter': 130, 'max_depth': 15, 'min_samples_leaf': 43, 'max_leaf_nodes': 13, 'l2_regularization': 0.015607945180858668}. Best is trial 0 with value: 0.9442874793983109.
[I 2024-10-10 13:19:28,695] Trial 2 finished with value: 0.9496629782084686 and parameters: {'learning_rate': 0.046633482231537286, 'max_iter': 164, 'max_depth': 9, 'min_samples_leaf': 43, 'max_leaf_nodes': 29, 'l2_regularization': 0.002394702316418791}. Best is trial 2 with val

Best parameters for HistGradientBoosting: {'learning_rate': 0.034645950382562865, 'max_iter': 475, 'max_depth': 11, 'min_samples_leaf': 45, 'max_leaf_nodes': 39, 'l2_regularization': 0.000795713415233366}


In [35]:
histgb_best_params = histgb_study.best_trial.params
histgb_best_score = histgb_study.best_trial.value
histgb_best_score

0.9520614132824511

## KNN

In [54]:
from sklearn.neighbors import KNeighborsClassifier

def knn_objective(trial):
    param = {
        'n_neighbors': trial.suggest_int('n_neighbors', 3, 15),
        'leaf_size': trial.suggest_int('leaf_size', 20, 50),
        'p': trial.suggest_categorical('p', [1, 2]),  # Minkowski distance parameter
        'weights': trial.suggest_categorical('weights', ['uniform', 'distance'])
    }

    model = KNeighborsClassifier(**param)
    score = cross_val_score(model, X_train_prep, y_train, cv=3, scoring='roc_auc').mean()
    return score

knn_study = optuna.create_study(direction='maximize')
knn_study.optimize(knn_objective, n_trials=50, timeout=600)
print('Best parameters for KNN:', knn_study.best_params)
print('Best score for KNN:', knn_study.best_value)

[I 2024-10-10 14:05:53,771] A new study created in memory with name: no-name-29719f91-dde2-4543-98ba-465659a61824
[I 2024-10-10 14:06:06,813] Trial 0 finished with value: 0.8876617762447964 and parameters: {'n_neighbors': 9, 'leaf_size': 25, 'p': 2, 'weights': 'distance'}. Best is trial 0 with value: 0.8876617762447964.
[I 2024-10-10 14:06:54,382] Trial 1 finished with value: 0.8819651385549676 and parameters: {'n_neighbors': 6, 'leaf_size': 32, 'p': 1, 'weights': 'uniform'}. Best is trial 0 with value: 0.8876617762447964.
[I 2024-10-10 14:07:09,623] Trial 2 finished with value: 0.8818342574413597 and parameters: {'n_neighbors': 7, 'leaf_size': 35, 'p': 2, 'weights': 'distance'}. Best is trial 0 with value: 0.8876617762447964.
[I 2024-10-10 14:07:24,234] Trial 3 finished with value: 0.8955094165429607 and parameters: {'n_neighbors': 15, 'leaf_size': 20, 'p': 2, 'weights': 'uniform'}. Best is trial 3 with value: 0.8955094165429607.
[I 2024-10-10 14:07:40,043] Trial 4 finished with value

Best parameters for KNN: {'n_neighbors': 15, 'leaf_size': 42, 'p': 1, 'weights': 'uniform'}
Best score for KNN: 0.8999919341832944


In [55]:
knn_best_params = knn_study.best_trial.params
knn_best_score = knn_study.best_trial.value
knn_best_score

0.8999919341832944

# Stacking

In [56]:
from sklearn.ensemble import StackingClassifier

# Init base models
xgb_model = xgb.XGBClassifier(**xgb_best_params)
cat_model = CatBoostClassifier(**cat_best_params)
lgb_model = lgb.LGBMClassifier(**lgb_best_params)
extratrees_model = ExtraTreesClassifier(**extratrees_best_params)
logreg_model = LogisticRegression(**logreg_best_params)
histgb_model = HistGradientBoostingClassifier(**histgb_best_params)
knn_model = KNeighborsClassifier(**knn_best_params)

# Base and meta models
base_estimators = [
	('xgb', xgb_model),
	('cat', cat_model),
	('lgb', lgb_model),
	('extratrees', extratrees_model),
	('logreg', logreg_model),
	('histgb', histgb_model),
	('knn', knn_model)
]

meta_model = LogisticRegression()

# Stack
stack_model = StackingClassifier(estimators=base_estimators, final_estimator=meta_model, cv=3, n_jobs=-1)
stack_model.fit(X_train_prep, y_train)

StackingClassifier(cv=3,
                   estimators=[('xgb',
                                XGBClassifier(alpha=4.17984861400515,
                                              base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=0.7817900881520917,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None,
                                              gamma=0.20733828465084803,
                                              grow_policy=None,
                                              im...
                                LogisticRegression(C=0.07290758788748607,
                                                   penalty='l1',
                                                   solver='saga')),
                               ('histgb',
                                HistGradientBoostingClassifier(l2_regularization=0.000795713415233366,
                                                               learning_rate=0.034645950382562865,
                                                               max_depth=11,
                                                               max_iter=475,
                                                               max_leaf_nodes=39,
                                                               min_samples_leaf=45)),
                               ('knn',
                                KNeighborsClassifier(leaf_size=42,
                                                     n_neighbors=15, p=1))],
                   final_estimator=LogisticRegression(), n_jobs=-1)

In [62]:
print('Stacking Classifier ROC AUC:', roc_auc_score(y_test, stack_model.predict_proba(X_test_prep)[:, 1]))
print('Stacking Classifier STD:', np.std(cross_val_score(stack_model, X_train_prep, y_train, cv=3, scoring='roc_auc')))

Stacking Classifier ROC AUC: 0.9599856509508792
Stacking Classifier STD: 0.0009124308654476986


In [63]:
# Choose different base models
base_estimators = [
	('xgb', xgb_model),
	('cat', cat_model),
	('lgb', lgb_model),
	('histgb', histgb_model),
	('knn', knn_model)
]

meta_model = lgb.LGBMClassifier()

# Stack
stack_model_2 = StackingClassifier(estimators=base_estimators, final_estimator=meta_model, cv=3, n_jobs=-1)
stack_model_2.fit(X_train_prep, y_train)

print('Stacking Classifier ROC AUC:', roc_auc_score(y_test, stack_model_2.predict_proba(X_test_prep)[:, 1]))

[LightGBM] [Info] Number of positive: 11630, number of negative: 58086
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004786 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1036
[LightGBM] [Info] Number of data points in the train set: 69716, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.166820 -> initscore=-1.608337
[LightGBM] [Info] Start training from score -1.608337
Stacking Classifier ROC AUC: 0.9603357103494445


# Submission

In [65]:
X_train_all = preprocessor.transform(X)

stack_model_2 = StackingClassifier(estimators=base_estimators, final_estimator=meta_model, cv=5, n_jobs=-1)
stack_model_2.fit(X_train_all, y)

[LightGBM] [Info] Number of positive: 14538, number of negative: 72608
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006130 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1036
[LightGBM] [Info] Number of data points in the train set: 87146, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.166823 -> initscore=-1.608309
[LightGBM] [Info] Start training from score -1.608309


StackingClassifier(cv=5,
                   estimators=[('xgb',
                                XGBClassifier(alpha=4.17984861400515,
                                              base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=0.7817900881520917,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None,
                                              gamma=0.20733828465084803,
                                              grow_policy=None,
                                              im...
                                               n_estimators=530, num_leaves=200,
                                               subsample=0.8031135623721948)),
                               ('histgb',
                                HistGradientBoostingClassifier(l2_regularization=0.000795713415233366,
                                                               learning_rate=0.034645950382562865,
                                                               max_depth=11,
                                                               max_iter=475,
                                                               max_leaf_nodes=39,
                                                               min_samples_leaf=45)),
                               ('knn',
                                KNeighborsClassifier(leaf_size=42,
                                                     n_neighbors=15, p=1))],
                   final_estimator=LGBMClassifier(), n_jobs=-1)

In [66]:
import joblib

joblib.dump(stack_model_2, '../models/stack_model_v2.pkl')

X_to_pred = preprocessor.transform(data_to_predict)

y_pred_submit = stack_model_2.predict_proba(X_to_pred)[:, 1]

submission = pd.DataFrame({'id': data_to_predict['id'], 'loan_status': y_pred_submit})
submission.to_csv('../submissions/stack_model_v2.csv', index=False)